<a href="https://colab.research.google.com/github/Nisag/EVA4/blob/master/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Session 4 Assignment:
1. 99.4% validation accuracy
2. Less than 20k parameters 
3. Less than 20 epochs
4. No fully connected layer

Things covered so far - 
* No of layers in a network 
* Receptive field
* 3x3 Convolutions 
* MaxPooling, position, distance from prediction 
* No of kernels 
* Image Normalization
* 1x1 Convolutions 
* Transition layers, concept, position
* SoftMax
* Batch Normalization, distance from prediction 
* Dropout, when to introduce 
* Larger kernel or alternatives - When to stop convolutions and go ahead 
* Performance - How do we know network is not doing well, comparatively, very early
* Batch Size, effects of batch size
* Learning Rate

To learn how to add different things we covered in this session, you can refer to this code: https://www.kaggle.com/enwei26/mnist-digits-pytorch-cnn-99 






## Import libraries

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## Model definition

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3) # input - 28*28*1, output - 26*26*8
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 3) # input - 26*26*8, output - 24*24*16
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 32, 3) # input - 24*24*16, output - 22*22*32
        self.bn3 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2) # input - 22*22*32, output - 11*11*32
        self.conv4 = nn.Conv2d(32, 8, 1) # input - 11*11*32, output - 11*11*8
        self.bn4 = nn.BatchNorm2d(8)
        self.conv5 = nn.Conv2d(8, 16, 3) # input - 11*11*8, output - 9*9*16
        self.bn5 = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 32, 3) # input - 9*9*16, output - 7*7*32
        self.bn6 = nn.BatchNorm2d(32)
        self.avgpool = nn.AdaptiveAvgPool2d(1) # input 7*7*32 output - 1*1*32
        self.conv7 = nn.Conv2d(32, 10, 1) # input - 1*1*32, output - 1*1*10

    def forward(self, x):
        x = self.bn2(F.relu(self.conv2(self.bn1(F.relu(self.conv1(x))))))
        x = self.pool1(self.bn3(F.relu(self.conv3(x))))
        x = self.bn5(F.relu(self.conv5(self.bn4(F.relu(self.conv4(x))))))
        x = self.bn6(F.relu(self.conv6(x)))
        x = self.avgpool(x)
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

## Model summary

In [26]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
       BatchNorm2d-2            [-1, 8, 26, 26]              16
            Conv2d-3           [-1, 16, 24, 24]           1,168
       BatchNorm2d-4           [-1, 16, 24, 24]              32
            Conv2d-5           [-1, 32, 22, 22]           4,640
       BatchNorm2d-6           [-1, 32, 22, 22]              64
         MaxPool2d-7           [-1, 32, 11, 11]               0
            Conv2d-8            [-1, 8, 11, 11]             264
       BatchNorm2d-9            [-1, 8, 11, 11]              16
           Conv2d-10             [-1, 16, 9, 9]           1,168
      BatchNorm2d-11             [-1, 16, 9, 9]              32
           Conv2d-12             [-1, 32, 7, 7]           4,640
      BatchNorm2d-13             [-1, 32, 7, 7]              64
AdaptiveAvgPool2d-14             [-1, 3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## Prepare and load the dataset

In [0]:
torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True,
                   download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


## Model training and validation

In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Optimizer definition and running the model

In [30]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0535, Accuracy: 9857/10000 (98.6%)



  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0347, Accuracy: 9900/10000 (99.0%)



  0%|          | 0/1875 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0193, Accuracy: 9940/10000 (99.4%)



  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0203, Accuracy: 9932/10000 (99.3%)



loss=0.0002491772174835205 batch_id=1874: 100%|██████████| 1875/1875 [00:20<00:00, 90.70it/s]



Test set: Average loss: 0.0211, Accuracy: 9938/10000 (99.4%)



### That's all Folks!